## **GF Project 5**

**Project Goal and Objectives**: Have the bot every 30 mins from 9am - 9pm EST give an update on the traffic flow and compare those values with MAs.

### **Bot**:
 - 1) Trigger the file request using a trigger: Trigger + Playwirght 
 - 2) Call the notebook (NEF_Project_5_local.ipynb)
 - 3) Output the print message from the notebook to the chat
 - 4) Keep in the directory the files shown in ORIGINAL_FILES list

### **NEF_Project_%_local.ipynb**:
- 1) Will start by impoting the file (Traffic View 3)
- 2) Calculate all the present stats 
- 3) Insert into the databse the values for the given day and hour 
- 4) Comapre result with MAs from same time of previous last week, if standard deviaiton >20% --> ALERT 

#### STEP 1 - Importing Traffic View for a given instance 

In [15]:
import os 
import pandas as pd

current_dir = os.getcwd()
files = os.listdir(current_dir)
for file in files:
    if "Project5" in file and file.endswith('.csv'):
        print(file)
        file_name = file
        
tview1 = pd.read_csv(file_name)
tview1.head()



Project5.csv


,Item,<track>Tracking 2</track>,Attempted Charges,Declined Charges,Declined Charge Rate,Successful Charges,Success Charge Rate,Unique Cycle Attempts,Declined Unique Cycle Attempts,Declined Unique Cycle Attempt Rate,...,Ad Spend,Ad Spend Rate,Chargeback Fees,Chargeback Fees Rate,Alert Fees,Alert Fees Rate,Processing Fees,Processing Fees Rate,Profit,Profit Margin
0,DP1 RP V3,Harley Davidson Evo Helmet,60,48,80.00%,12,20.00%,59,47,79.66%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$23.40,10.29%,$204.00,89.71%
1,DP1 RP V3,Harley Davidson Evo Helmet (black/orange),76,67,88.16%,9,11.84%,71,62,87.32%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$23.75,13.93%,$127.85,74.96%
2,DP1 RP V3,Heavy Duty Hitch Mount Flagpole Kit,10,9,90.00%,1,10.00%,10,9,90.00%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$2.95,15.57%,$16.00,84.43%
3,DP1 RP V3,Luminox Navy Seal Foundation Military Watch,6,4,66.67%,2,33.33%,6,4,66.67%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$3.10,8.18%,$34.80,91.82%
4,DP1 RP V3,Miller Lite Tailgate Combo Pack,68,58,85.29%,10,14.71%,61,51,83.61%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$23.10,12.19%,$166.40,87.81%


## First we will calulate teh value of the Prepaids

### **DP1 PP / (Dp1 pp + dp1)**

NOTE: This will be done on a per product basis

# 1)First we have to iterate throguh each product

In [16]:
# Filter only for products that have over 25 sales for the day 
filtered_df = tview1[(tview1["Successful Charges"] > 25) & (tview1["Item"] == "DP1 V3")]
valid_products = filtered_df["<track>Tracking 2</track>"].unique()
print(valid_products)

# Delete all rows that have RP in the Item column 
tview1 = tview1[~tview1["Item"].str.contains("RP")]


# keep tview1 for ony these products 
tview1 = tview1[tview1["<track>Tracking 2</track>"].isin(valid_products)]
tview1








['Heavy Duty Hitch Mount Flagpole Kit' 'Miller Lite Tailgate Combo Pack'
 'Rapala Fishing Tackle Bag']


,Item,<track>Tracking 2</track>,Attempted Charges,Declined Charges,Declined Charge Rate,Successful Charges,Success Charge Rate,Unique Cycle Attempts,Declined Unique Cycle Attempts,Declined Unique Cycle Attempt Rate,...,Ad Spend,Ad Spend Rate,Chargeback Fees,Chargeback Fees Rate,Alert Fees,Alert Fees Rate,Processing Fees,Processing Fees Rate,Profit,Profit Margin
17,DP1 V3,Heavy Duty Hitch Mount Flagpole Kit,264,169,64.02%,95,35.98%,160,65,40.63%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$112.56,9.19%,"$1,099.10",89.76%
23,DP1 V3,Miller Lite Tailgate Combo Pack,211,140,66.35%,71,33.65%,128,57,44.53%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$87.64,9.58%,$827.55,90.42%
28,DP1 V3,Rapala Fishing Tackle Bag,119,75,63.03%,44,36.97%,69,25,36.23%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$51.96,9.16%,$502.31,88.57%
38,DP1 V3(PP),Heavy Duty Hitch Mount Flagpole Kit,11,10,90.91%,1,9.09%,11,10,90.91%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$2.84,22.03%,$10.05,77.97%
42,DP1 V3(PP),Miller Lite Tailgate Combo Pack,13,10,76.92%,3,23.08%,13,10,76.92%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$4.52,11.69%,$34.15,88.31%
46,DP1 V3(PP),Rapala Fishing Tackle Bag,9,8,88.89%,1,11.11%,9,8,88.89%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$2.44,18.93%,$10.45,81.07%
72,DP12 V3,Heavy Duty Hitch Mount Flagpole Kit,26,22,84.62%,4,15.38%,15,11,73.33%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$17.82,4.51%,$377.74,95.49%
77,DP12 V3,Miller Lite Tailgate Combo Pack,139,119,85.61%,20,14.39%,83,63,75.90%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$110.46,5.58%,"$1,768.45",89.42%
79,DP12 V3,Rapala Fishing Tackle Bag,131,121,92.37%,10,7.63%,78,68,87.18%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$62.20,6.29%,$926.70,93.71%
90,DP12 V3(PP),Miller Lite Tailgate Combo Pack,4,4,100.00%,0,0.00%,4,4,100.00%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$0.80,0.00%,$-0.80,0.00%


In [17]:
unique_products = tview1["<track>Tracking 2</track>"].unique()
print(unique_products)

['Heavy Duty Hitch Mount Flagpole Kit' 'Miller Lite Tailgate Combo Pack'
 'Rapala Fishing Tackle Bag']


In [18]:
unique_charges = tview1["Item"].unique()
print(unique_charges)

['DP1 V3' 'DP1 V3(PP)' 'DP12 V3' 'DP12 V3(PP)']


In [19]:
# First step: Keep only rows that contain 'DP1'
tview1 = tview1[tview1["Item"].str.contains('DP1')]

# Second step: Remove any rows that contain 'DP12', 'DP13', etc
tview1 = tview1[~tview1["Item"].str.contains('DP1[2-9]')]
unique_charges = tview1["Item"].unique()
print(unique_charges)

['DP1 V3' 'DP1 V3(PP)']


In [20]:
import pytz
from datetime import datetime

ny_time = datetime.now(pytz.timezone('America/New_York')).strftime("%#I:%M%p").lower()
print(ny_time)

8:24am


In [28]:
message = f"**📺 TV message (NY {ny_time})**\n\n"
for product in unique_products:
    # Get product data
    df = tview1[tview1["<track>Tracking 2</track>"] == product]
    df_sucar = df[df["Item"] == 'DP1 V3']
    total_sales = df_sucar["Successful Charges"].sum()
    
    # Calculate Prepaid
    denom = df["Successful Charges"].sum()
    df_num = df[df["Item"].str.contains("PP")]
    num = df_num["Successful Charges"].sum()
    prepaid_percentage = 0 if num == 0 or denom == 0 else num / denom
    prepaid_percentage = prepaid_percentage * 100
    prepaid_percentage = round(prepaid_percentage, 2)
    
    # Get SUCAR and convert to numeric value for comparison
    sucar = df_sucar["Success Unique Cycle Attempt Rate"].values[0]
    sucar_value = float(sucar.strip('%')) if isinstance(sucar, str) else float(sucar)
    
    # Format product section
    message += f"**{product}**:\n"
    message += f"∘ Initials: {total_sales}\n"
    
    # Only show SUCAR value if no alert
    if sucar_value < 50:
        message += f"⚠️ **ALERT: SUCAR is below 50% ({sucar_value:.0f}%)**\n"
    else:
        message += f"▪ SUCAR: {sucar}\n"
        
    if prepaid_percentage == 0:
        message += f"▪ Prepaid: 0.00% (no data available)\n"
    else:
        if prepaid_percentage > 10:
            message += f"⚠️ **ALERT: Prepaids are above 10% ({prepaid_percentage:.2f}%)**\n"
        else:
            message += f"▪ Prepaid: {prepaid_percentage:.2f}%\n"
    message += "\n"

    
output_message = message

print(output_message)

**📺 TV message (NY 8:24am)**

**Heavy Duty Hitch Mount Flagpole Kit**:
∘ Initials: 95
▪ SUCAR: 59.38%
▪ Prepaid: 1.04%

**Miller Lite Tailgate Combo Pack**:
∘ Initials: 71
▪ SUCAR: 55.47%
▪ Prepaid: 4.05%

**Rapala Fishing Tackle Bag**:
∘ Initials: 44
▪ SUCAR: 63.77%
▪ Prepaid: 2.22%


